In [1]:
%%capture
!pip install datasets==1.14
!pip install transformers==4.19.0
!pip install librosa

!apt install git-lfs
!pip install -U datasets

In [20]:
import pandas as pd
from scipy.io import wavfile
import numpy as np
import datasets
import ast
import os
import numpy as np
import glob

from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/train/"
base_path_test = "/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
from datasets import load_dataset, load_metric, Dataset, Audio
metric = load_metric("accuracy")

In [22]:
drug_files_train = glob.glob(base_path+"drug/"+"*")
disease_files_train = glob.glob(base_path+"disease/"+"*")
treatment_files_train = glob.glob(base_path+"treatment/"+"*")

drug_files_test = glob.glob(base_path_test+"drug/"+"*")
disease_files_test = glob.glob(base_path_test+"disease/"+"*")
treatment_files_test = glob.glob(base_path_test+"treatment/"+"*")

In [23]:
disease_files_test

['/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording.wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (2).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (3).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (4).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (5).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (6).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (7).wav',
 '/content/drive/MyDrive/Colab Notebooks/IIT Kharagpur/drug-disease-treatment recordings/test/disease/Recording (8).wav',
 '/content/drive/MyDrive/Col

In [24]:
train_df = pd.DataFrame({"file" : drug_files_train + disease_files_train + treatment_files_train,
                   "label" : [0]*len(drug_files_train) + [1]*len(disease_files_train) + [2]*len(treatment_files_train)
})

test_df = pd.DataFrame({"file" : drug_files_test + disease_files_test + treatment_files_test,
                   "label" : [0]*len(drug_files_test) + [1]*len(disease_files_test) + [2]*len(treatment_files_test)
})

In [25]:
test_df

,file,label
0,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
1,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
2,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
3,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
4,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
5,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
6,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
7,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
8,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0
9,/content/drive/MyDrive/Colab Notebooks/IIT Kha...,0


In [26]:
def get_array_from_wav(filename):
    output = wavfile.read(filename)
    a = np.array(output[1],dtype=float) #output[0]#, the sampling rate
    return [sum(i)/2 for i in a]

In [27]:
train_df['audio'] = train_df['file'].apply(lambda x : get_array_from_wav(x))
test_df['audio'] = test_df['file'].apply(lambda x : get_array_from_wav(x))

In [28]:
train_df.to_csv('train.csv', index=False)
train_df = Dataset.from_csv('train.csv')

test_df.to_csv('test.csv', index=False)
test_df = Dataset.from_csv('test.csv')

Using custom data configuration default-0fa3b431f2410761


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0fa3b431f2410761/0.0.0. Subsequent calls will reuse this data.


Using custom data configuration default-eb112a0b2dca55ef


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-eb112a0b2dca55ef/0.0.0. Subsequent calls will reuse this data.


In [29]:
'''
train_df.cast_column("audio", Audio(sampling_rate=16000))
test_df.cast_column("audio", Audio(sampling_rate=16000))

dataset = datasets.DatasetDict({"train":train_df,"test":test_df})'''

'\ntrain_df.cast_column("audio", Audio(sampling_rate=16000))\ntest_df.cast_column("audio", Audio(sampling_rate=16000))\n\ndataset = datasets.DatasetDict({"train":train_df,"test":test_df})'

In [30]:
train_df.cast_column("audio", Audio(sampling_rate=16000))
test_df.cast_column("audio", Audio(sampling_rate=16000))

Dataset({
    features: ['file', 'label', 'audio'],
    num_rows: 60
})

In [31]:
train_df2 = {'file' : train_df['file'],
             'audio': [ast.literal_eval(i) for i in train_df['audio']],
             'label' : train_df['label']}
train_df = Dataset.from_dict(train_df2)
             
test_df2 = {'file' : test_df['file'],
             'audio': [ast.literal_eval(i) for i in test_df['audio']],
             'label' : test_df['label']}
test_df = Dataset.from_dict(test_df2)             

In [32]:
from datasets.dataset_dict import DatasetDict
from datasets.dataset_dict import DatasetDict
dataset={}
dataset["train"]=train_df
dataset["test"]=test_df
dataset=DatasetDict(dataset)

In [33]:
#dataset['train']['audio'][0]

In [34]:
id2label = {"1" : 1, "2" : 2, "3" : 3}
label2id = {1 : "1", 2 : "2", 3 : "3"}

In [44]:
from transformers import AutoFeatureExtractor
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32
max_duration = 1.5  # seconds
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

loading feature extractor configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/d4583dd9e59eb6295f8fe8b18833ae54d963a122d69aa1df7ecce6caafe18c8f.bc3155ca0bae3a39fc37fc6d64829c6a765f46480894658bb21c08db6155358d
loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a con

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [45]:
def preprocess_function(examples):
    audio_arrays = [x for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [46]:
preprocess_function(dataset['train'][:5])

{'input_values': [array([0.00417275, 0.00417275, 0.00417275, ..., 1.3826463 , 1.3770992 ,
       1.3881935 ], dtype=float32), array([0.00272125, 0.00272125, 0.00272125, ..., 0.9162121 , 0.92525655,
       0.8483786 ], dtype=float32), array([0.00030848, 0.00030848, 0.00030848, ..., 0.24232861, 0.14318785,
       0.07320611], dtype=float32), array([ 1.2791746e-04,  1.2791746e-04,  1.2791746e-04, ...,
       -1.3139033e+00, -1.4875292e+00, -1.6335326e+00], dtype=float32), array([ 0.00156908,  0.00156908,  0.00156908, ..., -1.0900412 ,
       -1.1662    , -1.3489813 ], dtype=float32)]}

In [47]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
encoded_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 210
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 60
    })
})

# Training/Finetuning

In [48]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreT

In [49]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-drug-disease-treatment",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


In [51]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [52]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 210
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 20


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.098478,0.283333
1,No log,1.085970,0.400000
2,No log,1.061995,0.616667
3,No log,1.042021,0.716667
4,No log,1.024316,0.700000
5,No log,1.007555,0.716667
6,No log,0.989731,0.716667
7,No log,0.973250,0.716667
8,No log,0.958921,0.733333
9,1.736600,0.943419,0.716667


***** Running Evaluation *****
  Num examples = 60
  Batch size = 32
Saving model checkpoint to wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-1
Configuration saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-1/config.json
Model weights saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-1/pytorch_model.bin
Feature extractor saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-1/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 60
  Batch size = 32
Saving model checkpoint to wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-2
Configuration saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-2/config.json
Model weights saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-2/pytorch_model.bin
Feature extractor saved in wav2vec2-base-finetuned-drug-disease-treatment/checkpoint-2/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 60
  Batch size = 32
Sav

TrainOutput(global_step=20, training_loss=1.6358736991882323, metrics={'train_runtime': 267.2743, 'train_samples_per_second': 15.714, 'train_steps_per_second': 0.075, 'total_flos': 5.6078866216224e+16, 'train_loss': 1.6358736991882323, 'epoch': 19.57})

In [53]:
#trainer.push_to_hub()

In [54]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 60
  Batch size = 32


{'epoch': 19.57,
 'eval_accuracy': 0.7833333333333333,
 'eval_loss': 0.8302195072174072,
 'eval_runtime': 1.0741,
 'eval_samples_per_second': 55.859,
 'eval_steps_per_second': 1.862}

Attribution:

https://github.com/manthanthakker/AudioClassification

https://huggingface.co/docs/datasets/package_reference/main_classes